In [3]:
import numpy as np
from sklearn.compose import ColumnTransformer
import pandas as pd
from nltk import sent_tokenize
from sklearn.pipeline import Pipeline,make_pipeline
import gensim
from gensim.utils import simple_preprocess

In [3]:
df=pd.read_csv('labelledData.csv.csv')

In [12]:
df=df.iloc[:,1:]

In [13]:
df.columns

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text',
       'sentiments'],
      dtype='object')

In [14]:
df=df.loc[:,['Text','sentiments']]

In [15]:
df = df[((df.index <= 60000) & (df['sentiments'] == 1)) | (df['sentiments'] == 0)]


In [16]:
df.to_csv('updatedlabelledData')

In [17]:
new_df=pd.read_csv('updatedlabelledData')

In [18]:
new_df=new_df.iloc[:,:]

In [19]:
new_df

,Unnamed: 0,Text,sentiments
0,0,I have bought several of the Vitality canned d...,1
1,1,Great taffy at a great price. There was a wid...,1
2,2,This saltwater taffy had great flavors and was...,1
3,3,This taffy is so good. It is very soft and ch...,1
4,4,Right now I'm mostly just sprouting this so my...,1
...,...,...,...
112264,415385,"I was disappointed in this product, as I had r...",0
112265,415386,"The candy is tasty, but they totally scam you ...",0
112266,415387,Definitely not worth buying flavored water wit...,0
112267,415388,I thought this soup would be more like a chill...,0


In [20]:
resutl=map(simple_preprocess,new_df.loc[:,'Text'].tolist())

In [21]:
preprocessed_result=list(resutl)


In [22]:
preprocessed_result

[['have',
  'bought',
  'several',
  'of',
  'the',
  'vitality',
  'canned',
  'dog',
  'food',
  'products',
  'and',
  'have',
  'found',
  'them',
  'all',
  'to',
  'be',
  'of',
  'good',
  'quality',
  'the',
  'product',
  'looks',
  'more',
  'like',
  'stew',
  'than',
  'processed',
  'meat',
  'and',
  'it',
  'smells',
  'better',
  'my',
  'labrador',
  'is',
  'finicky',
  'and',
  'she',
  'appreciates',
  'this',
  'product',
  'better',
  'than',
  'most'],
 ['great',
  'taffy',
  'at',
  'great',
  'price',
  'there',
  'was',
  'wide',
  'assortment',
  'of',
  'yummy',
  'taffy',
  'delivery',
  'was',
  'very',
  'quick',
  'if',
  'your',
  'taffy',
  'lover',
  'this',
  'is',
  'deal'],
 ['this',
  'saltwater',
  'taffy',
  'had',
  'great',
  'flavors',
  'and',
  'was',
  'very',
  'soft',
  'and',
  'chewy',
  'each',
  'candy',
  'was',
  'individually',
  'wrapped',
  'well',
  'none',
  'of',
  'the',
  'candies',
  'were',
  'stuck',
  'together',
  'whi

In [23]:
len(preprocessed_result)

112269

In [4]:
model=gensim.models.Word2Vec(
    window=10,
    min_count=5
)

In [12]:
from gensim.models import Word2Vec

# Load a pretrained Word2Vec model
model = Word2Vec.load('C:/Users/MSI 1/Desktop/Emotion Detection 2/Emotion Detection 2/wordEmbedding_model.gensim')

# Get vocabulary size
vocab_size = len(model.wv.key_to_index)

# Get vector dimensionality
vector_dimension = model.vector_size

# Estimate number of parameters for embeddings
embedding_parameters = vocab_size * vector_dimension

# Estimate number of parameters for hidden layer (skip-gram)
hidden_layer_size = 10  # Adjust this based on your model's architecture
hidden_layer_parameters = (vocab_size + vector_dimension) * hidden_layer_size

# Total number of parameters
total_parameters = embedding_parameters + hidden_layer_parameters

# Print the summary
print(f"Vocabulary Size: {vocab_size}")
print(f"Vector Dimensionality: {vector_dimension}")
print(f"Embedding Parameters: {embedding_parameters}")
print(f"Hidden Layer Parameters: {hidden_layer_parameters}")
print(f"Total Parameters: {total_parameters}")


Vocabulary Size: 20377
Vector Dimensionality: 100
Embedding Parameters: 2037700
Hidden Layer Parameters: 204770
Total Parameters: 2242470


In [10]:
print(gensim.models.Word2Vec())

Word2Vec<vocab=0, vector_size=100, alpha=0.025>


In [7]:
model.summary()

AttributeError: 'Word2Vec' object has no attribute 'summary'

In [24]:
model.build_vocab(preprocessed_result)

In [26]:
model.train(preprocessed_result,total_examples=model.corpus_count,epochs=model.epochs)

(32198981, 42481305)

In [ ]:
model.save("wordEmbedding_model.gensim")

In [38]:
model.wv.most_similar('hello')

[('president', 0.591320276260376),
 ('anywho', 0.5894930362701416),
 ('fellow', 0.5483019948005676),
 ('bless', 0.5369171500205994),
 ('butterworth', 0.5344894528388977),
 ('sally', 0.5320992469787598),
 ('flamingo', 0.5116368532180786),
 ('robbins', 0.5076295733451843),
 ('baskin', 0.49788692593574524),
 ('yikes', 0.4941539168357849)]

In [41]:
x=model.wv.get_normed_vectors()

AttributeError: 'KeyedVectors' object has no attribute 'models'

In [40]:
x.shape

(20377, 100)

In [ ]:
y = model.wv.index_to_key
y

In [48]:
len(y)

20377

In [51]:
model.wv.vector_size

100

In [2]:
def sent_vectorized(sentence):
	wv_res=np.zeros(model.wv.vector_size)
	ctr=1
	for word in sentence:
		if word in model.wv:
			ctr +=1
			wv_res+=model.wv[word]
	wv_res=wv_res/ctr
	return wv_res

In [77]:
result=map(sent_vectorized,preprocessed_result)

In [78]:
new_result=list(result)

In [1]:
new_result

NameError: name 'new_result' is not defined

In [91]:
new_result=np.array(new_result)

In [89]:
Y=new_df.iloc[:,1:]

In [90]:
y=np.array(Y)

In [171]:
from sklearn.model_selection import train_test_split

# Assuming you have your dataset as features (X) and labels (y)
# X is the feature matrix, and y is the target vector

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(new_result, y, test_size=0.3, random_state=42)

# Now you can use X_train and y_train to train your model
# And X_test to evaluate its performance


In [172]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score



# Create a logistic regression model
model = LogisticRegression(solver='lbfgs',max_iter=1000)

# Train the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")


c:\Users\MSI 1\Desktop\Emotion Detection 2\Emotion Detection 2\project\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.87


In [119]:
ytest=model.predict_proba(x1)

In [120]:
ytest

array([[0.26848628, 0.73151372]])

In [115]:
x1=(new_result[0]).reshape(1, -1)

In [116]:
x1.shape

(1, 100)

In [134]:
new_df.loc[0,'Text']

'I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.'

In [140]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score



# Create a Random Forest classifier
model = RandomForestClassifier(n_estimators=20, random_state=42)

# Train the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")


c:\Users\MSI 1\Desktop\Emotion Detection 2\Emotion Detection 2\project\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Accuracy: 0.87


In [143]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score



# Create an XGBoost classifier
xgb_model = xgb.XGBClassifier(n_estimators=20, random_state=42)

# Train the XGBoost model on the training data
xgb_model.fit(X_train, y_train)

# Make predictions using the XGBoost model
y_pred_xgb = xgb_model.predict(X_test)

# Calculate accuracy for XGBoost model
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
print(f"XGBoost Accuracy: {accuracy_xgb:.2f}")


XGBoost Accuracy: 0.87


In [147]:

from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense


In [173]:

# Create a neural network model with hidden layers
nn_model = Sequential()
nn_model.add(Dense(16, input_dim=100, activation='relu'))  # First hidden layer with input_dim=100
nn_model.add(Dense(8, activation='relu'))                 # Second hidden layer
nn_model.add(Dense(1, activation='sigmoid'))              # Output layer

# Compile the model
nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the neural network model on the training data
nn_model.fit(X_train, y_train, epochs=10, batch_size=16, verbose=0)


In [174]:

# Make predictions using the neural network model
y_pred_nn = nn_model.predict(X_train)
custom_threshold=0.5
y_pred_classes = (y_pred_nn > custom_threshold).astype(int)
# Calculate accuracy for the neural network model
accuracy_nn = accuracy_score(y_train, y_pred_classes)
print(f"Neural Network Accuracy: {accuracy_nn:.2f}")


2456/2456 [==============================] - 2s 790us/step
Neural Network Accuracy: 0.90


In [175]:
# Make predictions using the neural network model
y_pred_nn = nn_model.predict(X_test)
custom_threshold=0.5
y_pred_classes = (y_pred_nn > custom_threshold).astype(int)
# Calculate accuracy for the neural network model
accuracy_nn = accuracy_score(y_test, y_pred_classes)
print(f"Neural Network Accuracy: {accuracy_nn:.2f}")

1053/1053 [==============================] - 1s 946us/step
Neural Network Accuracy: 0.89


In [169]:
len(y_test)

22454

In [20]:
new_df

,Text,sentiments
0,I have bought several of the Vitality canned d...,1
1,Great taffy at a great price. There was a wid...,1
2,This saltwater taffy had great flavors and was...,1
3,This taffy is so good. It is very soft and ch...,1
4,Right now I'm mostly just sprouting this so my...,1
...,...,...
112264,"I was disappointed in this product, as I had r...",0
112265,"The candy is tasty, but they totally scam you ...",0
112266,Definitely not worth buying flavored water wit...,0
112267,I thought this soup would be more like a chill...,0


# With pipeline


In [112]:
from sklearn.preprocessing import StandardScaler,FunctionTransformer


In [56]:
scaler=StandardScaler()

In [106]:
import nltk
from nltk.tokenize import word_tokenize

In [ ]:
df

In [4]:
from sklearn.model_selection import train_test_split

# Assuming you have your dataset as features (X) and labels (y)
# X is the feature matrix, and y is the target vector

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['Text'], df['sentiments'], test_size=0.3, random_state=42)

In [7]:
processed=[simple_preprocess(text) for text in X_train]


In [8]:
processed

[['drink',
  'rooibos',
  'tea',
  'because',
  'its',
  'caffeine',
  'free',
  'great',
  'mild',
  'taste',
  'and',
  'has',
  'beautiful',
  'clear',
  'red',
  'color',
  'its',
  'good',
  'quality',
  'product',
  'at',
  'good',
  'price',
  'now',
  'that',
  'know',
  'the',
  'quality',
  'of',
  'davidson',
  'product',
  'am',
  'going',
  'to',
  'try',
  'different',
  'tea',
  'products',
  'from',
  'them'],
 ['authentic',
  'strawberry',
  'scent',
  'and',
  'flavor',
  'very',
  'tasty',
  'both',
  'hot',
  'or',
  'iced',
  'the',
  'best',
  'one',
  'in',
  'the',
  'ahmad',
  'lineup',
  'of',
  'flavored',
  'teas',
  'not',
  'because',
  'fan',
  'of',
  'strawberry',
  'fruit',
  'but',
  'because',
  'this',
  'tea',
  'is',
  'mild',
  'tasting',
  'yet',
  'full',
  'of',
  'flavor',
  'while',
  'the',
  'other',
  'ahmad',
  'varieties',
  'have',
  'less',
  'authentic',
  'flavor',
  'and',
  'leave',
  'generic',
  'bitter',
  'taste',
  'in',
  't

In [151]:
y_pred_nn

array([[0.3316257 ],
       [0.00609427],
       [0.99466914],
       ...,
       [0.2513901 ],
       [0.34166965],
       [0.09622721]], dtype=float32)